In [2]:
import pandas as pd

df = pd.read_csv('transactional-sample.csv')

print(df.head())

df.info()

   transaction_id  merchant_id  user_id       card_number  \
0        21320398        29744    97051  434505******9116   
1        21320399        92895     2708  444456******4210   
2        21320400        47759    14777  425850******7024   
3        21320401        68657    69758  464296******3991   
4        21320402        54075    64367  650487******6116   

             transaction_date  transaction_amount  device_id  has_cbk  
0  2019-12-01T23:16:32.812632              374.56   285475.0    False  
1  2019-12-01T22:45:37.873639              734.87   497105.0     True  
2  2019-12-01T22:22:43.021495              760.36        NaN    False  
3  2019-12-01T21:59:19.797129             2556.13        NaN     True  
4  2019-12-01T21:30:53.347051               55.36   860232.0    False  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----

In [3]:
# Check for unique values in the date column
unique_date = df['transaction_date'].unique()
print(unique_date)

# Or simply
df['transaction_date'].nunique() == len(df)

['2019-12-01T23:16:32.812632' '2019-12-01T22:45:37.873639'
 '2019-12-01T22:22:43.021495' ... '2019-11-01T10:23:50.555604'
 '2019-11-01T01:29:45.799767' '2019-11-01T01:27:15.811098']


True

In [4]:
# Convert data types
df['card_number'] = df['card_number'].astype(str)
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   transaction_id      3199 non-null   int64         
 1   merchant_id         3199 non-null   int64         
 2   user_id             3199 non-null   int64         
 3   card_number         3199 non-null   object        
 4   transaction_date    3199 non-null   datetime64[ns]
 5   transaction_amount  3199 non-null   float64       
 6   device_id           2369 non-null   float64       
 7   has_cbk             3199 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 178.2+ KB


In [5]:
has_chargeback = df['has_cbk'].sum()
print(has_chargeback)

391


In [6]:
df_filtered = df.dropna(subset=['device_id'])
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2369 entries, 0 to 2920
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   transaction_id      2369 non-null   int64         
 1   merchant_id         2369 non-null   int64         
 2   user_id             2369 non-null   int64         
 3   card_number         2369 non-null   object        
 4   transaction_date    2369 non-null   datetime64[ns]
 5   transaction_amount  2369 non-null   float64       
 6   device_id           2369 non-null   float64       
 7   has_cbk             2369 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 150.4+ KB


In [7]:
# Check if all device_id values have a zero decimal
all_zero_decimal = df_filtered['device_id'].apply(lambda x: str(x).endswith('.0')).all()
print(all_zero_decimal)

equals_minus_one = df_filtered['device_id'].apply(lambda x: x == -1).all()
print(equals_minus_one)

True
False


In [8]:
# Convert valid values to integer (coerce for NaN) and use sentinel value for NaN
def convert_floats_to_int(df):
    converted = pd.to_numeric(df['device_id'], errors='coerce')
    converted = converted.fillna(-1)  
    converted = converted.astype(int)

    # Assign the converted column back (avoids SettingWithCopy warning)
    df['device_id'] = converted
    return df

df = df.pipe(convert_floats_to_int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   transaction_id      3199 non-null   int64         
 1   merchant_id         3199 non-null   int64         
 2   user_id             3199 non-null   int64         
 3   card_number         3199 non-null   object        
 4   transaction_date    3199 non-null   datetime64[ns]
 5   transaction_amount  3199 non-null   float64       
 6   device_id           3199 non-null   int32         
 7   has_cbk             3199 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int32(1), int64(3), object(1)
memory usage: 165.7+ KB


In [9]:
mean_transaction_amount = df['transaction_amount'].mean()
print(mean_transaction_amount)

767.8129040325101


In [13]:
count_high_amount_transactions = (df['transaction_amount'] > 4000).sum()
print("transactions with high amount (> 3000):",count_high_amount_transactions)

high_amount_transactions = df[df['transaction_amount'] > 4000]
high_amount_transactions

transactions with high amount (> 3000): 41


,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk
465,21320863,49909,73385,553636******3778,2019-11-29 22:06:50.179099,4043.09,884678,True
667,21321065,61529,25385,512267******3121,2019-11-29 13:53:13.284000,4043.96,305010,False
684,21321082,15676,54976,439354******6004,2019-11-29 13:16:16.106153,4078.80,-1,False
690,21321088,72057,28518,415275******5134,2019-11-29 13:02:57.328217,4067.90,310277,False
790,21321188,60526,17929,606282******9429,2019-11-28 22:20:04.236179,4055.23,960729,True
912,21321310,64830,3442,400437******3137,2019-11-28 17:44:54.561463,4034.56,-1,False
920,21321318,18493,75771,444458******3957,2019-11-28 17:30:08.316714,4080.03,288748,False
1025,21321423,61529,25385,512267******3121,2019-11-28 11:27:22.259566,4070.87,498185,False
1149,21321547,91972,24644,650486******2500,2019-11-26 20:22:41.390309,4072.90,977260,True
1260,21321658,65330,78262,553636******6300,2019-11-25 17:30:49.997119,4058.92,547440,True


In [11]:
# Sort the DataFrame
df_sorted = df.sort_values(['user_id', 'transaction_date'])

# Calculate the time difference within each group
df_sorted['time_diff'] = df_sorted.groupby('user_id')['transaction_date'].diff()

# Check if any differences are less than 2 minutes
transactions_within_2min = df_sorted[df_sorted['time_diff'] < pd.Timedelta(minutes=2)]

transactions_within_2min

,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk,time_diff
3123,21323521,4039,266,482425******1320,2019-11-03 20:25:23.212894,235.70,-1,False,0 days 00:00:33.173242
450,21320848,23591,1836,444456******3799,2019-11-29 23:12:53.556465,708.34,889989,False,0 days 00:01:48.439832
1814,21322212,71378,9853,527407******8091,2019-11-22 18:57:08.043119,19.82,686249,True,0 days 00:01:07.654588
964,21321362,46045,10378,415944******1540,2019-11-28 15:41:40.107253,553.66,17372,True,0 days 00:00:57.319523
2573,21322971,44459,10405,515894******4290,2019-11-17 01:06:14.821040,258.46,856642,False,0 days 00:00:33.445949
3102,21323500,76725,16781,546056******2924,2019-11-04 14:55:21.764584,502.16,-1,True,0 days 00:00:32.744905
2751,21323149,55531,17407,498453******3618,2019-11-12 01:09:51.468180,2726.27,-1,False,0 days 00:01:01.437758
196,21320594,81795,17807,606282******2612,2019-11-30 20:24:00.906638,568.42,288714,True,0 days 00:01:00.267273
3108,21323506,62052,18227,550209******3098,2019-11-04 13:17:29.295709,941.39,-1,True,0 days 00:01:38.304480
2228,21322626,53041,28218,498406******7104,2019-11-20 21:42:21.655846,426.84,174844,True,0 days 00:01:28.091811


In [12]:
# Sort the DataFrame
df_sorted = df.sort_values(['user_id', 'transaction_date', 'transaction_amount'])

# Calculate the time and amount difference within each group
df_sorted['time_diff'] = df_sorted.groupby('user_id')['transaction_date'].diff() 
df_sorted['amount_diff'] = df_sorted.groupby('user_id')['transaction_amount'].diff()

# Check if any time differences are less than 5 minutes and the corresponding amount differences are 10
transactions_within_5min_diff_10 = df_sorted[(df_sorted['time_diff'] <= pd.Timedelta(minutes=10)) & (df_sorted['amount_diff'].abs() < 10)]

transactions_within_5min_diff_10

# df_filtered = df[df['user_id'] == 81152]
# df_filtered

,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk,time_diff,amount_diff
2814,21323212,43078,2866,535081******3568,2019-11-10 23:42:41.103589,165.82,-1,False,0 days 00:06:18.562176,-2.56
2650,21323048,99679,6024,410863******8215,2019-11-15 18:35:48.628972,10.79,701116,False,0 days 00:03:27.875193,5.40
1816,21322214,71378,9853,527407******8091,2019-11-22 18:56:00.388531,22.95,686249,True,0 days 00:06:05.485805,3.78
1814,21322212,71378,9853,527407******8091,2019-11-22 18:57:08.043119,19.82,686249,True,0 days 00:01:07.654588,-3.13
2573,21322971,44459,10405,515894******4290,2019-11-17 01:06:14.821040,258.46,856642,False,0 days 00:00:33.445949,-3.47
1638,21322036,68380,13001,650901******4110,2019-11-23 11:53:17.948760,10.55,715437,False,0 days 00:04:11.730976,-0.36
2943,21323341,77130,42677,515601******8618,2019-11-08 20:41:36.944243,301.58,-1,True,0 days 00:00:18.161919,-2.24
3132,21323530,57964,50105,606282******4832,2019-11-03 18:34:54.311401,2.42,-1,False,0 days 00:09:47.167363,-0.14
1104,21321502,55854,53850,527468******1757,2019-11-27 14:52:06.001713,471.06,20098,True,0 days 00:02:13.381403,-0.04
3113,21323511,32901,62541,511781******250,2019-11-04 11:22:22.321651,65.35,-1,False,0 days 00:01:40.794694,0.64
